In [1]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras import layers
from keras.layers import Input, Add, Dense, Dropout, Activation, ZeroPadding2D, BatchNormalization, Concatenate, Flatten, Conv2D, AveragePooling2D, MaxPool2D, Reshape, Conv2DTranspose
import os
import random
import numpy as np
from skimage.io import imread
from skimage.transform import resize
import matplotlib.pyplot as plt
from tqdm import tqdm 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

2023-05-16 16:30:21.036941: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [18]:
!python3 -c 'import tensorflow as tf; print(tf.__version__)'

2023-05-16 17:15:22.101918: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2.4.1


In [2]:
# Set seed for reproducibility
seed = 42
random.seed(seed)
tf.random.set_seed(seed)

In [4]:
#!git clone https://github.com/femartip/Nuclei-Segmentation.git

Cloning into 'Nuclei-Segmentation'...
remote: Enumerating objects: 42082, done.
remote: Counting objects: 100% (2/2), done.
remote: Total 42082 (delta 0), reused 2 (delta 0), pack-reused 42080
Receiving objects: 100% (42082/42082), 372.82 MiB | 11.29 MiB/s, done.
Resolving deltas: 100% (11025/11025), done.
Updating files: 100% (33893/33893), done.


In [5]:
#%cd Nuclei-Segmentation/

/content/Nuclei-Segmentation


# Load Images

In [3]:
TRAIN_PATH = './processed_data/train/'
TRAIN_MASK_PATH = './processed_data/train_mask/'
VAL_PATH = './processed_data/val/'
VAL_MASK_PATH = './processed_data/val_mask/'

X_train_arrays = []
for filename in os.listdir(TRAIN_PATH):
    image = Image.open(os.path.join(TRAIN_PATH, filename))
    image_array = np.array(image)
    image_array = np.expand_dims(image_array, axis=-1)
    X_train_arrays.append(image_array)
X_train = np.stack(X_train_arrays, axis=0)

X_val_arrays = []
for filename in os.listdir(VAL_PATH):
    image = Image.open(os.path.join(VAL_PATH, filename))
    image_array = np.array(image)
    image_array = np.expand_dims(image_array, axis=-1)
    X_val_arrays.append(image_array)
X_val = np.stack(X_val_arrays, axis=0)

y_train_arrays = []
for filename in os.listdir(TRAIN_MASK_PATH):
    image = Image.open(os.path.join(TRAIN_MASK_PATH, filename))
    image_array = np.array(image)
    bool_array = image_array != 0
    bool_array = np.expand_dims(bool_array, axis=-1)
    y_train_arrays.append(bool_array)
y_train = np.stack(y_train_arrays, axis=0)

y_val_arrays = []
for filename in os.listdir(VAL_MASK_PATH):
    image = Image.open(os.path.join(VAL_MASK_PATH, filename))
    image_array = np.array(image)
    bool_array = image_array != 0
    bool_array = np.expand_dims(bool_array, axis=-1)
    y_val_arrays.append(bool_array)
y_val = np.stack(y_val_arrays, axis=0)

In [10]:
print(X_train.shape)
print(y_train.shape)

print(X_val.shape)
print(y_val.shape)

(26232, 128, 128, 1)
(26232, 128, 128, 1)
(3028, 128, 128, 1)
(3028, 128, 128, 1)


# Model

In [11]:
import keras.backend as K

def dice_coef(y_true, y_pred, smooth=1.0):
    y_true_f = K.flatten(K.cast(y_true, 'float32'))
    y_pred_f = K.flatten(K.cast(y_pred, 'float32'))
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f+y_pred_f) + smooth)

In [12]:
def dice_coef_loss(y_true, y_pred):
    return 1.0-dice_coef(y_true, y_pred)

In [13]:
def iou(y_true, y_pred):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3]) - intersection
    iou = K.mean((intersection + 1) / (union + 1), axis=0)
    return iou

In [14]:
def mean_iou(y_true, y_pred):
    results = []   
    for t in np.arange(0.5, 1, 0.05):
        t_y_pred = tf.cast((y_pred > t), tf.float32)
        pred = iou(y_true, t_y_pred)
        results.append(pred)
        
    return K.mean(K.stack(results), axis=0)

In [15]:
def plot(history):
    plt.figure(figsize=(15,5))
    plt.subplot(1,2,1)
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='validation')
    plt.title('Loss')
    plt.legend()
    plt.subplot(1,2,2)
    plt.plot(history.history['mean_iou'], label='train')
    plt.plot(history.history['val_mean_iou'], label='validation')
    plt.title('Mean IoU')
    plt.legend()
    plt.show()

In [19]:
#from google.colab import drive
#drive.mount('/content/drive')
#!ls

Mounted at /content/drive
data  data_exploration.ipynb  model.ipynb  README.md


In [30]:
model = tf.keras.models.load_model("pre-trained_model.h5", compile=False)

In [33]:
count = 0
for layer in model.layers:
    if layer.name == 'block5_pool':
        print(count)
        break
    count += 1

19


In [31]:
model.layers[19].trainable = True

In [34]:
batch_size = 16
batches_per_epoch = int(X_train.shape[0]+X_val.shape[0]/batch_size) 
epochs = 5
LR = 0.00001

model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR), loss=dice_coef_loss, metrics=[mean_iou])

history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val), callbacks=tf.keras.callbacks.TensorBoard(log_dir='./logs'))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 128, 128, 3)  0           input_1[0][0]                    
                                                                 input_1[0][0]                    
                                                                 input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 128, 128, 64) 1792        concatenate[0][0]                
______________________________________________________________________________________________

2023-05-16 17:49:07.081271: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2023-05-16 17:49:07.081337: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2023-05-16 17:49:07.083341: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1365] Profiler found 1 GPUs
2023-05-16 17:49:07.099219: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcupti.so.10.1
2023-05-16 17:49:07.548803: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2023-05-16 17:49:07.549038: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1487] CUPTI activity buffer flushed
2023-05-16 17:49:07.556952: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 429785088 exceeds 10% of free system memory.
2023-05-16 17:49:07.871643: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 429785088 exceeds 10% of free system memory.
2023

Epoch 1/5


2023-05-16 17:49:11.693011: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2023-05-16 17:49:14.552740: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2023-05-16 17:49:14.800516: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-05-16 17:49:18.584059: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-05-16 17:49:21.656911: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.54GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


   2/1640 [..............................] - ETA: 6:04 - loss: 0.0607 - mean_iou: 0.8985    

: 

: 

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model,to_file='model_.png',show_shapes=True, show_layer_names=True)
from IPython.display import Image
Image(retina=True, filename='model_.png')

In [ ]:
print("Final accuracy on train set:", history.history['mean_iou'][-1])
print("Final loss on train set:", history.history['loss'][-1])
print("Final accuracy on validations set:", history.history['val_mean_iou'][-1])
print("Final loss on validations set:", history.history['val_loss'][-1])

In [ ]:
plot(history)

In [ ]:
idx = random.randint(0, len(X_train))

preds_train = model.predict(X_train[idx:idx+1], verbose=1)

preds_train_t = (preds_train > 0.5).astype(np.uint8)

plt.figure(figsize=(20,10))
plt.subplot(1,3,1)
plt.title('Original image')
plt.imshow(X_train[idx])
plt.subplot(1,3,2)
plt.title('Ground truth mask')
plt.imshow(np.squeeze(y_train[idx]))
plt.subplot(1,3,3)
plt.title('Predicted mask')
plt.imshow(np.squeeze(preds_train_t))
plt.show()